<a href="https://colab.research.google.com/github/currencyfxjle/Knapsack_1.1_valid/blob/main/Knapsack_GDF_Original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client

In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 12.4 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk Model - Volatility of financial variables.ipynb
Directory: /content/drive/My Drive/Colab Notebooks
  Sub-directory: GDF - Knapsack
  File: GDF - Risk Model 1.1
  File: GDF - Risk Model 1.2
  File: Constants.py   Calculator update
  File: GDF - Knapsack Problem .ipynb
  File: GDF

In [9]:
import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

File exists: /content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet


# **Original Knapsack**

In [30]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['project_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')
    supply_df = supply_df.dropna()
    supply_df = supply_df.sort_values(by=['project_date'], ascending=True)

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['allocated_amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['pending_amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Sort purchasers by Total Appetite descending
    demand_df = demand_df.sort_values(by='approximated_appetite', ascending=False)
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['pending_amount']
        total_appetite = purchaser['approximated_appetite']
        allocated_to_purchaser = purchaser['allocated_amount']

        for jdx, project in supply_df.iterrows():
            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['project_date']

            # Check constraints: full project cost fits the remaining pending amount and respects deadlines
            if (project_cost <= remaining_pending_amount and
                total_appetite >= allocated_to_purchaser + project_cost and
                not project['Fully Allocated'] and
                project_date > purchaser['effective_date_purchaser']):  # Strict constraint

                # Allocate project fully
                allocations.append((
                    purchaser['purchaser_name'],
                    project_id,
                    project_name,
                    project['Original Est Cash Amount'],  # Original Est Cash Amount
                    project_cost,  # Allocated Amount
                    total_appetite,
                    total_appetite - (allocated_to_purchaser + project_cost),  # Remaining Appetite
                    allocated_to_purchaser + project_cost,  # Updated Allocated Amount
                    remaining_pending_amount - project_cost,  # Updated Pending Amount
                    purchaser['effective_date_purchaser'].date(),
                    project_date.date(),
                    "Yes" if project_cost == project['Original Est Cash Amount'] else "No"  # Fully Allocated Flag
                ))
                remaining_pending_amount -= project_cost
                allocated_to_purchaser += project_cost
                total_allocated += project_cost
                supply_df.at[jdx, 'Est Cash Amount'] = 0  # Mark project as fully allocated
                supply_df.at[jdx, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0:
                    break

        # Update purchaser's allocated_amount and pending_amount in demand_df
        demand_df.at[idx, 'allocated_amount'] = allocated_to_purchaser
        demand_df.at[idx, 'pending_amount'] = remaining_pending_amount

        # Update unallocated demand
        if remaining_pending_amount > 0:
            unallocated_demand.append((
                purchaser['purchaser_name'],
                remaining_pending_amount,  # Remaining Pending Amount
                purchaser['effective_date_purchaser'].date()
            ))

    return total_allocated, allocations, unallocated_demand, supply_df

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        # Allocations Tab
        pd.DataFrame(allocations, columns=[
            "Purchaser", "Project ID", "Project Name", "Original Est Cash Amount", "Allocated Amount",
            "Total Appetite", "Remaining Appetite", "Allocated Amount Updated", "Pending Amount",
            "Effective Date Purchaser", "Project Date", "Fully Allocated"
        ]).to_excel(writer, sheet_name="Allocations", index=False)

        # Summary Tab
        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)

        # Unallocated Demand Tab
        pd.DataFrame(unallocated_demand, columns=["Purchaser", "Pending Amount", "Effective Date Purchaser"]).to_excel(
            writer, sheet_name="Unallocated Demand", index=False
        )

        # Remaining Supply Tab
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, unallocated_demand, supply_df = match_supply_to_demand(supply_df, demand_df)

# Calculations for Summary
total_pending_amount = demand_df['pending_amount'].sum()
total_project_value = supply_df['Original Est Cash Amount'].sum()
remaining_supply_value = supply_df['Est Cash Amount'].sum()
total_unallocated_value = sum([pending for _, pending, _ in unallocated_demand])

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Total Project Value", "Value": total_project_value},
    {"Metric": "Remaining Supply Value", "Value": remaining_supply_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
    {"Metric": "Total Unallocated Value", "Value": total_unallocated_value}
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    unallocated_demand,
    supply_df[['Project ID', 'Proyect Name', 'Est Cash Amount', 'Fully Allocated', 'project_date']],
    "allocation_results.xlsx"
)

# Results Summary
print(f"Total Project Value (Original): {total_project_value}")
print(f"Total Pending Amount (After Allocations): {total_pending_amount}")
print(f"Total Value Allocated: {total_allocated}")
print(f"Remaining Supply Value: {remaining_supply_value}")
print(f"Total Unallocated Value: {total_unallocated_value}")


Results saved to: allocation_results.xlsx
Total Project Value (Original): 44025757.75
Total Pending Amount (After Allocations): 41782964.79
Total Value Allocated: 23301180.31
Remaining Supply Value: 20724577.440000005
Total Unallocated Value: 41782964.79


# **ADDING x1.25**

In [33]:
import pandas as pd

# Load results from the previously saved Excel file
def load_previous_results(file_name):
    """Load Remaining Supply and Unallocated Demand from Excel"""
    remaining_supply = pd.read_excel(file_name, sheet_name="Remaining Supply")
    unallocated_demand = pd.read_excel(file_name, sheet_name="Unallocated Demand")

    # Check column names for debugging
    print("Unallocated Demand Columns:", unallocated_demand.columns)
    print("Remaining Supply Columns:", remaining_supply.columns)

    # Normalize column names (handle spaces, case sensitivity, etc.)
    unallocated_demand.columns = unallocated_demand.columns.str.strip().str.lower().str.replace(' ', '_')
    remaining_supply.columns = remaining_supply.columns.str.strip().str.lower().str.replace(' ', '_')

    # Add columns for initial remaining value and increased pending amount by 1.25
    unallocated_demand['initial_pending_amount'] = unallocated_demand['pending_amount']
    unallocated_demand['pending_amount_1_25'] = unallocated_demand['pending_amount'] * 1.25

    return remaining_supply, unallocated_demand

# Create new allocations based on remaining supply and unallocated demand
def create_new_allocations(remaining_supply, unallocated_demand):
    """Allocate remaining supply to unallocated demand"""
    new_allocations = []

    for idx, purchaser in unallocated_demand.iterrows():
        remaining_pending_amount = purchaser['pending_amount_1_25']  # Use 1.25 increased pending amount

        for jdx, project in remaining_supply.iterrows():
            if project['fully_allocated']:
                continue

            project_cost = project['est_cash_amount']
            project_date = project['project_date']

            # Check constraints: Effective Date Purchaser must be before Project Date
            if project_cost <= remaining_pending_amount and project_date > purchaser['effective_date_purchaser']:
                # Allocate project only if demand can fully cover the supply
                if remaining_pending_amount >= project_cost:
                    allocated_amount = project_cost
                    new_allocations.append({
                        "Purchaser": purchaser['purchaser'],
                        "Project ID": project['project_id'],
                        "Project Name": project['proyect_name'],
                        "Allocated Amount": allocated_amount,
                        "Initial Pending Amount": purchaser['initial_pending_amount'],
                        "Pending Amount (1.25x)": purchaser['pending_amount_1_25'],
                        "Remaining Pending Amount": remaining_pending_amount - allocated_amount,
                        "Effective Date Purchaser": purchaser['effective_date_purchaser'],
                        "Project Date": project['project_date'],
                    })
                    remaining_pending_amount -= allocated_amount
                    remaining_supply.at[jdx, 'est_cash_amount'] -= allocated_amount
                    remaining_supply.at[jdx, 'fully_allocated'] = remaining_supply.at[jdx, 'est_cash_amount'] == 0

                    if remaining_pending_amount <= 0:
                        break

    return new_allocations

# Save the updated results to a new Excel file
def save_results(new_allocations, remaining_supply, unallocated_demand, filename):
    """Save results to an Excel file"""
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        # Allocations Tab
        pd.DataFrame(new_allocations).to_excel(writer, sheet_name="Allocations", index=False)
        # Unallocated Demand Tab
        unallocated_demand.to_excel(writer, sheet_name="Unallocated Demand", index=False)
        # Remaining Supply Tab
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)

    print(f"Results saved to: {filename}")

# Main execution for creating new allocations
remaining_supply, unallocated_demand = load_previous_results("allocation_results.xlsx")

new_allocations = create_new_allocations(remaining_supply, unallocated_demand)

# Save the updated results to a new Excel file
save_results(
    new_allocations,
    remaining_supply,
    unallocated_demand,
    "allocation_results_round2.xlsx"
)

# Print summary of allocations
print(f"Total Allocations Created: {len(new_allocations)}")


Unallocated Demand Columns: Index(['Purchaser', 'Pending Amount', 'Effective Date Purchaser'], dtype='object')
Remaining Supply Columns: Index(['Project ID', 'Proyect Name', 'Est Cash Amount', 'Fully Allocated',
       'project_date'],
      dtype='object')
Results saved to: allocation_results_round2.xlsx
Total Allocations Created: 9
